<a href="https://colab.research.google.com/github/rabeyatahir/Cryptography-and-Applications/blob/main/Crypto_Assessment_ECDSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ECDSA Algorithm Implementation
This notebook aims to cover the implementation of ECDSA(Elliptic Curve Digital Signature Algorithm). In this regard, the notebook is divided into the following sections:

1. Key generation
2. Signature assigning against files or folders
3. Signature verification for the files and folders

# ECDSA Algorithm Steps:

1. Key Generation
  * Elliptic Curve Key Pair Generation:Generate a new key pair using the P-256 elliptic curve.
  * Export Keys: Exporting the private and public keys in PEM format.
  * Folder Structure: Creating a folder structure to organize keys based on user and date.
  * Storing Keys: Saving the generated keys in the specified folder.

2. Signing files for Encryption
  * Read Private Key: Read the private key from the specified file.
  * Import Private Key: Import the private key into the ECDSA algorithm.
  * Hash Calculation:Calculate the SHA-256 hash of the file contents.
  * Signing: Sign the hash using the ECDSA algorithm, producing a digital signature.

3. Folder Signing
  * Iterate Through Files:Iterate through each file in the specified folder.
  * Sign Each File: For each file, obtain the digital signature.
  * Save Signatures: Save the signatures in separate files with a specific extension.

4. Verification for files
  * Read Public Key: Read the public key from the specified file.
  * Import Public Key: Import the public key into the ECDSA algorithm.
  * Hash Calculation: Calculate the SHA-256 hash of the received message.
  * Verification: Verify the digital signature using the sender's public key and the calculated hash.

5. Folder Signature Verification
  * Iterate Through Signature Files: Iterate through each file in the specified folder with a specific extension indicating a signature file.
  * Verify Each Signature:For each signature file, verify the corresponding file's signature.

These steps illustrate the key processes involved in ECDSA, including key generation, signing, and verification. The algorithm provides a secure way to ensure message integrity and authenticity.

## Advantages of using ECDSA to RSA

As mentioned [here](https://www.leaderssl.com/articles/484-rsa-ecc-ecdsa-which-algorithm-is-better-to-choose-when-ordering-a-digital-certificate-in-leaderssl), using ECDSA for digital signatures offers several key benefits, including:

1. Enhanced security measures.
2. Uninterrupted application performance.
3. Swift signing and verification processes (40% faster compared to RSA).
4. Adherence to evolving application security standards.
5. Alignment with governmental information protection regulations.
6. Compliance with contemporary industry requirements.

Furthermore, certificates employing ECDSA can effectively minimize the overall volume of data requiring authentication, leading to substantial cost savings associated with data storage.

## Installing Dependency Packages

In [ ]:
%pip install pycryptodome==3.19.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.5 MB/s eta 0:00:00


##Step 1: Generation of Private/Public Keys for Two Different Users

In [ ]:
"""# Mounting google drive to access folders
from google.colab import drive
drive.mount('/content/drive')"""

In [ ]:

# Importing libraries
from Crypto.PublicKey import ECC
import os
import datetime

def generate_key_pair(user_name):
      """
        Description:
            Generates a digital signature key pair for the specified user and saves it in a designated folder structure.

        Args:
            user_name (str): The name of the user for which we want to generate the key pair

      """
      # Generating an elliptic curve key pair using P-256 prime curve
      key = ECC.generate(curve='P-256')

      # Exporting the private and public keys in PEM format
      private_key = key.export_key(format='PEM')
      public_key = key.public_key().export_key(format='PEM')

      # Creating a folder for each user to store public and private keys
      user_folder = f"user_keys/{user_name}"

      # Check if the user_keys folder already exists, then do not raise an error
      os.makedirs(user_folder, exist_ok=True)

      # Get the current date for key rotation to manage keys
      current_date = datetime.date.today()

      # Check if keys for the current date already exist
      key_folder = os.path.join(user_folder, str(current_date))

      # Check if the key_folder already exists, then do not raise an error
      os.makedirs(key_folder, exist_ok=True)

      # Defination of paths for saving private and public keys
      """
          Here, key_folder represents the directory based on the current date with filenames that include the
          name of the user and the type of key (private or public) with the extension .pem.
      """
      private_key_path = os.path.join(key_folder, f"private_key_{user_name}.pem")
      public_key_path = os.path.join(key_folder, f"public_key_{user_name}.pem")

      # Saving the new public and private keys as bytes
      with open(private_key_path, "wb") as f:
          f.write(private_key.encode('utf-8'))
      with open(public_key_path, "wb") as f:
          f.write(public_key.encode('utf-8'))
      print("\n==================================================================")
      print(f"New key pair for {user_name} generated and saved in {key_folder}.")
      print("\n==================================================================")

if __name__ == "__main__":

  # Taking input usernames
  user1 = input("Enter first username for which you want to generate public and private keys: ")
  user2 = input("Enter second username for which you want to generate public and private keys: ")

  # Calling generate_key_pair to generate keys for both users against specified date
  generate_key_pair(user1)
  generate_key_pair(user2)

Enter first username for which you want to generate public and private keys: Rabeya
Enter second username for which you want to generate public and private keys: Tahir

New key pair for Rabeya generated and saved in user_keys/Rabeya/2024-01-17.


New key pair for Tahir generated and saved in user_keys/Tahir/2024-01-17.



##Step 2: Assigning Digital Signature for Encryption

In [ ]:
from Crypto.PublicKey import ECC
from Crypto.Signature import DSS
from Crypto.Hash import SHA256
import os

def sign_file(file_path, private_key_file):
    """
    Description:
        Sign the contents of a file using ECDSA.

    Args:
        file_path (str): Path to the file to be signed, here we can pass .txt, .pdf , .docx files
        private_key_file (str): String representing the private key file path.

    Returns:
       signature (bytes): Returns digital signature in bytes.
    """
    # Reading the contents of the private_key_file
    with open(private_key_file, "r") as f:
        private_key = f.read()

    # Importing the private key
    key = ECC.import_key(private_key)

    # Creating a SHA-256 hash object and update it with the contents
    h = SHA256.new()

    """
        Updating SHA-256 hash object (h) with the contents of a file specified by file_path in chunks
    """
    # Reading in binary mode
    with open(file_path, "rb") as f:

        # Iterating through the file in chunks of 8192 bytes
        while chunk := f.read(8192):

            # Updating the SHA-256 hash object with the current chunk of data
            h.update(chunk)

    # Creating a DSS signer object and sign the hash
    """
      Here,'fips-186-3' specifies the version of the Digital Signature Standard (DSS) and
      'der' specifies the encoding format for the signature Distinguished Encoding Rules (DER).
    """
    signer = DSS.new(key, 'fips-186-3', 'der')
    signature = signer.sign(h)
    return signature

def sign_files_in_folder(folder_path, private_key_file):
    """
    Description:
        Sign all files in a folder using ECDSA.

    Args:
        folder_path (str): Path to the folder containing files to be signed.
        private_key_file (str): String representing the private key file path.
    """
    # Iterate through files in the specified folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Checking if the item is a file
        if os.path.isfile(file_path):

            # Signing the file and save the signature file with specified extension
            signature = sign_file(file_path, private_key_file)
            signature_file_path = file_path + ".signature"

            with open(signature_file_path, "wb") as f:
                f.write(signature)
            print(f"Signature for {filename} saved in {signature_file_path}")

if __name__ == "__main__":

    # Taking user input for file or folder path, private key, and user name
    input_path = input("Enter file or folder path: ")
    private_key_file = input("Enter private key file path: ")

    # Check if input_path is a directory folder
    if os.path.isdir(input_path):

        # Sign all files in the folder and add .signature at the end of each file
        sign_files_in_folder(input_path, private_key_file)

    # Check if input_path is a file
    elif os.path.isfile(input_path):

        # Sign the specified file
        signature = sign_file(input_path, private_key_file)
        signature_file_path = input_path + ".signature"

        # Writing signature in a file
        with open(signature_file_path, "wb") as f:
            f.write(signature)
        print(f"Signature for {input_path} saved in {signature_file_path}")
    else:
        print(f"Error: {input_path} is not a valid file or folder.")


Enter file or folder path: /content/test.pdf
Enter private key file path: /content/user_keys/Rabeya/2024-01-17/private_key_Rabeya.pem
Signature for /content/test.pdf saved in /content/test.pdf.signature


##Step 3: Verification of digital Signature

In [ ]:
from Crypto.PublicKey import ECC
from Crypto.Signature import DSS
from Crypto.Hash import SHA256
import os

def verify_signature(file_path, signature, public_key_file):
    """
    Decription:
      Verifying the digital signature of a file using ECDSA to decrypt a file with contents

    Args:
        file_path (str): Path to the file to be verified.
        signature (bytes): Digital signature in bytes.
        public_key_file (str): Path to the public key file.

    Returns:
        bool: True if the signature is valid, False otherwise.
    """
    # Read the contents of the public_key_file
    with open(public_key_file, "rb") as f:
        public_key = f.read()

    # Importing the public key
    key = ECC.import_key(public_key)

    # Creating a SHA-256 hash object and update it with the contents
    h = SHA256.new()

    # Reading in binary mode
    with open(file_path, "rb") as f:
        # Iterating through the file in chunks of 8192 bytes
        while chunk := f.read(8192):
            # Updating the SHA-256 hash object with the current chunk of data
            h.update(chunk)

    # Creating a DSS verifier object and verify the hash against the signature
    verifier = DSS.new(key, 'fips-186-3', 'der')

    """
      Here, it verifies whether the provided digital signature (signature) corresponds
      to the hash value (h) of the data that was signed. If the verification is successful, it returns True
      else False
    """
    try:
      # Verification of the signature using the Digital Signature Standard (DSS) verifier
        verifier.verify(h, signature)
        return True

     # If verification fails, catch the exception and return False
    except ValueError:
        return False

def verify_signatures_in_folder(folder_path, public_key_file):
    """
    Description:
        Verification of digital signatures of files in a folder using ECDSA.

    Args:
        folder_path (str): Path to the folder containing files with signatures.
        public_key_file (str): Path to the public key file.
    """
    # Iterate through files in the specified folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Checking if the item is a file and ends with ".signature"
        if os.path.isfile(file_path) and filename.endswith(".signature"):

            # Construct the path to the signature file
            signature_file_path = file_path

            # Remove ".signature" from the filename to get the original file path
            file_path = file_path[:-10]

            # Read the contents of the signature file
            with open(signature_file_path, "rb") as f:
                signature = f.read()

            # Verify the signature validity for the original file
            is_valid = verify_signature(file_path, signature, public_key_file)
            if is_valid:
                print(f"Signature for {filename} is valid.")
            else:
                print(f"Signature for {filename} is not valid.")

if __name__ == "__main__":

    # Taking user input for file or folder path, public key, and user name
    input_path = input("Enter file or folder path: ")
    public_key_file = input("Enter public key file path: ")  # Ensure this input is a string

    # Check if input_path is a directory folder
    if os.path.isdir(input_path):
        # Verify signatures of all files in the folder
        verify_signatures_in_folder(input_path, public_key_file)

    # Check if input_path is a file
    elif os.path.isfile(input_path):
        # Construct the path to the signature file
        signature_file_path = input_path + ".signature"

        # Check if the signature file exists
        if os.path.exists(signature_file_path):
            # Read the contents of the signature file
            with open(signature_file_path, "rb") as f:
                signature = f.read()

            # Verify the signature for the specified file
            is_valid = verify_signature(input_path, signature, public_key_file)
            if is_valid:
                print(f"Signature for {input_path} is valid.")
            else:
                print(f"Signature for {input_path} is not valid.")
        else:
            print(f"Error: Signature file not found for {input_path}.")
    else:
        print(f"Error: {input_path} is not a valid file or folder.")


Enter file or folder path: /content/test.pdf.signature
Enter public key file path: /content/user_keys/Rabeya/2024-01-17/public_key_Rabeya.pem
Error: Signature file not found for /content/test.pdf.signature.


# Summarized Findings

Following is the overall ECDSA workflow:

1. Key Generation: Generate a key pair consisting of a private key and a corresponding public key.

2. Signature Generation for Encryption:

  * Choosing a file with message to be signed.
  * Applying hashing to the file using a secure hash function such as SHA-256.
  * Applying signature to the hash value with the user's private key to generate a digital signature.

3. Signature Verification for Decryption:

  * Applying hashing to the original file using the same hash function.
  * Verifying the signature using the hash value, the received signature, and the sender's public key.
  
4. Results: Displaying the result of the signature verification.

